<a href="https://colab.research.google.com/github/snig-17/QM-GRUNGE/blob/main/claire_peak_expeditions_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv("/Grunge/everest_expeditions.csv")

# Extract year from 'Yr/Seas'
df["Year"] = df["Yr/Seas"].str[:4].astype(int)

In [ ]:
exp_per_year = (
    df.groupby("Year")["Exped ID"]
      .nunique()
      .reset_index(name="Expeditions")
)


In [ ]:
df["Smtres"] = pd.to_numeric(df["Smtres"], errors="coerce")

exp_scale = (
    df.groupby("Year")["Smtres"]
      .sum(min_count=1)
      .reset_index(name="Total_Summiters")
)


In [ ]:
df_mem = pd.read_excel("/Grunge/Peak Deaths Report, members (1990-2025).xlsx")
df_hir = pd.read_excel("/Grunge/Peak deaths report, Hired (1990-2025).xlsx")



In [ ]:

df_mem.columns
df_hir.columns

Index(['Peak', 'Name', 'Yr/Seas', 'Date', 'Time', 'Citizenship', 'Sex', 'Age',
       'Oxy', 'Smt', 'Cause of Death'],
      dtype='object')

In [ ]:
df_mem["Year"] = df_mem["Yr/Seas"].str[:4].astype(int)
df_hir["Year"] = df_hir["Yr/Seas"].str[:4].astype(int)


In [ ]:
client_deaths = (
    df.groupby("Year")
      .size()
      .reset_index(name="Client_deaths")
)


In [ ]:
client_deaths = (
    df_mem
    .groupby("Year")
    .size()
    .reset_index(name="Client_deaths")
)

sherpa_deaths = (
    df_hir
    .groupby("Year")
    .size()
    .reset_index(name="Sherpa_deaths")
)


In [ ]:
sherpa_deaths = (
    df.groupby("Year")
      .size()
      .reset_index(name="Sherpa_deaths")
)


In [ ]:
sherpa_deaths.head()


,Year,Sherpa_deaths
0,1990,2
1,1992,5
2,1993,1
3,1994,2
4,1995,3


In [ ]:
client_deaths.head()

,Year,Client_deaths
0,1990,2
1,1991,2
2,1992,3
3,1993,6
4,1994,3


In [ ]:
# ====== 0) Imports ======
import pandas as pd
import numpy as np

# ====== 1) Load files (EDIT paths if your folder is different) ======
exp_path = "/Grunge/everest_expeditions.csv"
mem_path = "/Grunge/Peak Deaths Report, members (1990-2025).xlsx"
hir_path = "/Grunge/Peak deaths report, Hired (1990-2025).xlsx"

df_exp = pd.read_csv(exp_path)
df_mem = pd.read_excel(mem_path)
df_hir = pd.read_excel(hir_path)

# ====== 2) Create Year column in each dataframe ======
df_exp["Year"] = df_exp["Yr/Seas"].astype(str).str[:4].astype(int)
df_mem["Year"] = df_mem["Yr/Seas"].astype(str).str[:4].astype(int)
df_hir["Year"] = df_hir["Yr/Seas"].astype(str).str[:4].astype(int)

# ====== 3) Exposure measures from expedition data ======
# 3A) number of expeditions per year
exp_per_year = (
    df_exp.groupby("Year")["Exped ID"]
          .nunique()
          .reset_index(name="Expeditions")
)

# 3B) total summiters per year (optional exposure proxy)
df_exp["Smtres"] = pd.to_numeric(df_exp.get("Smtres"), errors="coerce")
smt_per_year = (
    df_exp.groupby("Year")["Smtres"]
          .sum(min_count=1)
          .reset_index(name="Total_Summiters")
)

# merge the exposure pieces
exposure = exp_per_year.merge(smt_per_year, on="Year", how="left")

# ====== 4) Deaths per year (clients vs sherpas) ======
client_deaths = (
    df_mem.groupby("Year")
          .size()
          .reset_index(name="Client_deaths")
)

sherpa_deaths = (
    df_hir.groupby("Year")
          .size()
          .reset_index(name="Sherpa_deaths")
)

# ====== 5) Build the analysis table ======
analysis_df = (
    exposure
    .merge(client_deaths, on="Year", how="left")
    .merge(sherpa_deaths, on="Year", how="left")
)

# Fill missing deaths with 0 (years with no recorded deaths)
analysis_df[["Client_deaths", "Sherpa_deaths"]] = (
    analysis_df[["Client_deaths", "Sherpa_deaths"]].fillna(0).astype(int)
)

# Keep years in your question window
analysis_df = analysis_df[(analysis_df["Year"] >= 1990) & (analysis_df["Year"] <= 2025)]

# ====== 6) Compute risks + relative risk ======
# Choose your exposure column:
# - "Expeditions" (safe default)
# - OR "Total_Summiters" (if you prefer intensity)
EXPOSURE_COL = "Expeditions"

# Avoid divide-by-zero just in case
analysis_df = analysis_df[analysis_df[EXPOSURE_COL].fillna(0) > 0].copy()

analysis_df["Client_risk"] = analysis_df["Client_deaths"] / analysis_df[EXPOSURE_COL]
analysis_df["Sherpa_risk"] = analysis_df["Sherpa_deaths"] / analysis_df[EXPOSURE_COL]

# Relative risk: Sherpa risk divided by client risk
analysis_df["Relative_risk"] = analysis_df["Sherpa_risk"] / analysis_df["Client_risk"]

# Optional: handle years where client_risk == 0 (Relative_risk becomes inf)
analysis_df["Relative_risk"] = analysis_df["Relative_risk"].replace([np.inf, -np.inf], np.nan)

# ====== 7) Remove shock years (optional, but you said you’ve been excluding these) ======
shock_years = [2015, 2020, 2021, 2022]
analysis_df_clean = analysis_df[~analysis_df["Year"].isin(shock_years)].copy()

# Optional: rolling average of relative risk to smooth noise
analysis_df_clean["RR_3yr_avg"] = analysis_df_clean["Relative_risk"].rolling(3, min_periods=1).mean()

# ====== 8) Quick check: show the final table ======
analysis_df_clean.head(15)

# (Optional) Save to CSV so you can reuse it easily
analysis_df_clean.to_csv("/content/everest_analysis_table.csv", index=False)
print("Saved: /content/everest_analysis_table.csv")




Saved: /content/everest_analysis_table.csv


In [ ]:
# ====== 9) Plot (optional, but you’ll almost certainly want this) ======
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.plot(analysis_df_clean["Year"], analysis_df_clean["Relative_risk"], label="Relative risk (raw)", alpha=0.6)
plt.plot(analysis_df_clean["Year"], analysis_df_clean["RR_3yr_avg"], label="Relative risk (3yr avg)", linewidth=2)
plt.axhline(1, linestyle="--")
plt.xlabel("Year")
plt.ylabel("Sherpa risk / Client risk")
plt.title(f"Relative Fatality Risk on Everest Over Time (Exposure = {EXPOSURE_COL})")
plt.legend()
plt.show()
